# Run circuits in device emulators

# Implement Noise Model

In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, Aer
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
import pandas as pd
from qiskit.tools.jupyter import *
import warnings
# warnings.filterwarnings('ignore')
from qiskit import *
import qiskit
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals, QuantumInstance
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, tensored_meas_cal,
                                                 CompleteMeasFitter, TensoredMeasFitter)

/tmp/ipykernel_48450/2432490180.py:19: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import (complete_meas_cal, tensored_meas_cal,


In [2]:
from qiskit.providers.fake_provider import *

In [31]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
shot = int(np.floor(1800000 / 631))
# shot = 10


In [32]:
shot

2852

In [33]:
shot * 631

1799612

## Hamiltonian conversion

By using SHVQE, we need to apply the inversal of Heisenberg part to the original Hamiltonian. First, load the original Hamiltonian from `Hamiltonian/OHhamiltonian.txt`.

In [5]:
with open('Hamiltonian/OHhamiltonian.txt', 'r') as file:
    pauli_text_lines = file.readlines()
paulis = []
weight = []
for line in pauli_text_lines:
    line = line.replace(' ', '')
    coeff, pauli_text_string = line.split("*")
    coeff = float(coeff)
    weight.append(coeff)
    # pauli_text_string = pauli_text_string[::-1] # TODO: this is a quick but wrong fix for qubits order
    pauli_text_string = pauli_text_string.replace('\n', '')
    paulis.append(pauli_text_string)

Then, we use `Stim` to apply the inversal of Heisenberg part to Hamiltonian. In our protocal, the Heisenberg part consists of `cz` gates, which commute with each other. The inversal of the Heisenberg part is itself.

In [6]:
import stim

Load Heisenberg circuit.

In [7]:
heisenberg_text_combined = ''

with open('saved_models/shvqe_clifford_ncz0_hei.qasm', 'r') as file:
    heisenberg_text_lines = file.readlines()
    heisenberg_text_combined = ''.join(heisenberg_text_lines[3:]) # omit the header

In [8]:
import re


# Define the regular expression pattern
pattern = r"q\[(\d+)\],q\[(\d+)\]"

# Define the replacement string
replacement = r"\1 \2"

# Use the sub() function to replace the matches
stim_text = re.sub(pattern, replacement, heisenberg_text_combined).replace(';', '')

Apply the Heisenberg circuit to each Pauli string.

In [9]:
%%time

heisenberg_circ = stim.Circuit(stim_text)
paulis_new = []

for p in paulis:
    ps  = stim.PauliString(p.replace('I', '_'))
    ps = ps.after(heisenberg_circ)
    paulis_new.append(ps.__str__().replace('_', 'I'))

CPU times: user 23.4 ms, sys: 0 ns, total: 23.4 ms
Wall time: 22.7 ms


Construct the new Hamiltonian as observable. The number of Pauli strings is not changed by Clifford.

In [10]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp(paulis_new, weight)
print(f">>> Observable size: {observable.size}")

>>> Observable size: 631


# Circuit Transpile and Execution

In [11]:
from qiskit import qasm2
from estimator_rem import EstimatorRem
from qiskit_aer.primitives import Estimator

circuit = qasm2.load('saved_models/shvqe_clifford_ncz0_sch.qasm')

Reverse qubits to respect qiskit convention.

In [12]:
circuit = circuit.reverse_bits()

In [14]:
# circuit.draw('mpl');

Transpile the circuit based on given system model from IBMQ_Montreal. **We use default transpiler from qiskit.**

In [13]:
system_model = FakeMontreal()

In [16]:
# transpiled_circuit = transpile(circuit, backend=system_model)

In [17]:
# transpiled_circuit.draw();

In [18]:
# qr = qiskit.QuantumRegister(12)
# cr = ClassicalRegister(12)
# qubit_list = [[0],[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11]]
# circuits_mea = circuit.copy()
# circuits_mea.measure_all()
# # circuits_mea.draw()
# cal_circuits, mit_pattern = tensored_meas_cal(mit_pattern=qubit_list, qr=qr, circlabel='mcal')


## Noiseless
Evaluate on the qiskit Estimator. Consider noiseless first.

In [19]:
estimator_noiseless = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': None # noiseless
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False,
    
)

In [20]:
# %%time
job_noiseless = estimator_noiseless.run(circuit, observable)
result_noiseless = job_noiseless.result()
# print(f">>> {result}")
result_noiseless.values

In [15]:
def get_error_rate(val):
    ref_value = -74.38714627
    nuclear_repulsion_energy = 4.365374966545
    energy_with_nuclear_repulsion = val + nuclear_repulsion_energy
    error_rate = abs(abs(ref_value - energy_with_nuclear_repulsion) / ref_value * 100)
    return error_rate

In [ ]:
get_error_rate(result_noiseless.values)[0]

0.10027378508814073

In [23]:

noise_model_name = ['fakecairo', 'fakekolkata', 'fakemontreal']
measure_noise_matrix = {}
representations = {}
for noise_name in noise_model_name:
    measure_noise_matrix[noise_name] = []
    noise_dict = pd.read_pickle('./NoiseModel/'+noise_name+'.pkl')


    single_qubit_noise_operator = []
    # two_qubits_noise_operator = []
    # two_graphs = [] # the map for cx

    for qubit in range(27):
        single_qubit_noise_operator.append([])
        
    for noise_1 in noise_dict['errors']:
        if len(noise_1['gate_qubits'][0])==1:
            qubit = noise_1['gate_qubits'][0][0]
            single_qubit_noise_operator[qubit].append(noise_1)
        # elif len(noise['gate_qubits'][0])==2:
        #     two_graphs.append(noise['gate_qubits'][0])
        #     two_qubits_noise_operator.append(noise)
            
    # print(len(single_qubit_noise_operator))
    # print(len(single_qubit_noise_operator[0]))
    # print(len(two_qubits_noise_operator))

 

    single_sup_op_start = time.time()
    # id, sx, x, rz's, sx*x, sx*rz's, x*rz's, 
    single_noisy_super_op_list = []
    for qubit in range(27):
        single_noisy_super_op_list.append([])

        for noisy_operator in single_qubit_noise_operator[qubit]:
            if noisy_operator['operations'][0]=='measure':
                measure_noise_matrix[noise_name].append(np.array(noisy_operator['probabilities']).T)

In [ ]:
# measure_noise_matrix['fakecairo']

## Noise models

### Fake Kolkata

In [16]:
import pickle

In [26]:
read_out_noise_matrix_Fake_Kolkata = measure_noise_matrix['fakekolkata'][0:12]
read_out_noise_matrix_Fake_Kolkata_inv = []
print(len(read_out_noise_matrix_Fake_Kolkata))
for i in range(len(read_out_noise_matrix_Fake_Kolkata)):
    matrix = read_out_noise_matrix_Fake_Kolkata[i]
    read_out_noise_matrix_Fake_Kolkata_inv.append(np.linalg.inv(matrix))
    
# print(read_out_noise_matrix_Fake_Kolkata_inv)
# t = []
# for i in range(len(read_out_noise_matrix_Fake_Kolkata)):
#     t.append(read_out_noise_matrix_Fake_Kolkata_inv[i] @ read_out_noise_matrix_Fake_Kolkata[i])
# print(t)

12


In [27]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model_fakekolkata = noise.NoiseModel.from_dict(pickle.load(file))

/tmp/ipykernel_48450/3864961774.py:2: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model_fakekolkata = noise.NoiseModel.from_dict(pickle.load(file))


In [ ]:
# qr = qiskit.QuantumRegister(12)
# cr = ClassicalRegister(12)
# qubit_list = [[0],[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11]]
# circuits_mea = circuit.copy()
# circuits_mea.measure_all()
# # circuits_mea.draw()
# cal_circuits, mit_pattern = tensored_meas_cal(mit_pattern=qubit_list, qr=qr, circlabel='mcal')
# backend = qiskit.Aer.get_backend('qasm_simulator')
# job_mea = qiskit.execute(cal_circuits, backend=backend, shots=10000, noise_model=noise_model_fakekolkata)
# cal_results = job_mea.result()

# meas_fitter = TensoredMeasFitter(cal_results, mit_pattern, circlabel='mcal')
# meas_filter = meas_fitter.filter


In [19]:
# noise_model_fakekolkata

In [34]:
# options = Options()
# options.resilience_level = 0
# qi_fakekolkata = QuantumInstance(backend=qiskit.Aer.get_backend('qasm_simulator'), 
#                      backend_options = {
#         # simulation options
#         'method': 'density_matrix',
#         'device': 'CPU',
#         'max_parallel_threads' : 0,
#         'max_parallel_shots' : 0,
#         'statevector_parallel_threshold' : 5,
#         'coupling_map' : system_model.configuration().coupling_map,
#         'noise_model': noise_model_fakekolkata,
#     },
#                      seed_simulator=seeds, seed_transpiler=seed_transpiler,
#                          coupling_map=system_model.configuration().coupling_map, noise_model=noise_model_fakekolkata,
#                          measurement_error_mitigation_cls=TensoredMeasFitter)
estimator_fakekolkata = EstimatorRem(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': noise_model_fakekolkata,
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False,
    measurement_noise_matrix_inv = read_out_noise_matrix_Fake_Kolkata_inv
    # options = options
)

In [ ]:
# result_fakekolkata = qiskit_algorithms.eval_observables(qi_fakekolkata, circuit, observable)

In [35]:
# %%time

job_fakekolkata = estimator_fakekolkata.run(circuit, observable)
result_fakekolkata = job_fakekolkata.result()
# mitigated_results = meas_filter.apply(result_fakekolkata,method == 'pseudo_inverse')


result_fakekolkata.values[0]

> /home/muzhou/codes/QC-Contest-underwater/estimator_rem.py(591)_pauli_expval_with_variance()
    589 
    590 
--> 591 
    592     variances = 1 - expvals**2
    593     return expvals, variances



In [31]:
get_error_rate(result_fakekolkata.values)[0]

0.8323550570372368

### Fake Cairo

In [27]:
with open('NoiseModel/fakecairo.pkl', 'rb') as file:
    noise_model_fakecairo = noise.NoiseModel.from_dict(pickle.load(file))

/tmp/ipykernel_24570/1409486957.py:2: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model_fakecairo = noise.NoiseModel.from_dict(pickle.load(file))


In [31]:
estimator_fakecairo = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': noise_model_fakecairo
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [32]:
%%time

job_fakecairo = estimator_fakecairo.run(circuit, observable)
result_fakecairo = job_fakecairo.result()
result_fakecairo.values[0]

CPU times: user 1h 55s, sys: 36.1 s, total: 1h 1min 31s
Wall time: 1min 2s


-78.15958989384637

In [33]:
get_error_rate(result_fakecairo.values)[0]

0.7970884385677267

### Fake Montreal

In [34]:
with open('NoiseModel/fakemontreal.pkl', 'rb') as file:
    noise_model_fakemontreal = noise.NoiseModel.from_dict(pickle.load(file))

/tmp/ipykernel_24570/3624331980.py:2: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model_fakemontreal = noise.NoiseModel.from_dict(pickle.load(file))


In [35]:
noise_model_fakemontreal

<NoiseModel on ['', 'measure', 'sx', 'id', 'reset', 'x', 'cx']>

In [36]:
estimator_fakemontreal = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': noise_model_fakemontreal
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [37]:
%%time

job_fakemontreal = estimator_fakemontreal.run(circuit, observable)
result_fakemontreal = job_fakemontreal.result()
result_fakemontreal.values[0]

CPU times: user 1h 1min 14s, sys: 36.3 s, total: 1h 1min 51s
Wall time: 1min


-77.52843855466759

In [38]:
get_error_rate(result_fakemontreal.values)[0]

1.6455567167940766

## Duration

Obtain the Duration of Quantum Circuit

In [39]:
from qiskit import pulse

Do remember to set the optimization_level to 0 if your circuit is already transpiled!!!

In [40]:
transpiled_circuit = transpile(circuit, backend=system_model)

with pulse.build(system_model) as program:
  with pulse.transpiler_settings(optimization_level=0):
    pulse.call(transpiled_circuit)

In [41]:
program.duration

320